In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the directories for source images and where to move them
source_dir = '/content/drive/MyDrive/Processed_IMG_NEW'
processed_dir = '/content/drive/MyDrive/Processed_IMG_NEW'


# Create train and test directories
train_dir = '/content/drive/MyDrive/Processed_IMG_NEW/train'
test_dir = '/content/drive/MyDrive/Processed_IMG_NEW/test'

Resnet 50 Batch size 16 and epochs 25

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import mixed_precision

# Enable mixed precision for faster computation
mixed_precision.set_global_policy('mixed_float16')

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 224  # ResNet50 default
IMG_WIDTH = 224
BATCH_SIZE = 16
EPOCHS = 25  # Increased for better feature learning
PATIENCE = 25

# Image Data Generator with proper preprocessing for ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # ADDED
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2  # ADDED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

#Convert class_weights to dictionary
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

#Adjust unfreezing
for layer in base_model.layers[:-50]:  # Instead of -80
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr * 1.2  # Gradually increase LR in first 5 epochs
    elif epoch < 10:
        return lr  # Keep stable
    else:
        return lr * 0.95  # Reduce gradually

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=5, factor=0.3, min_lr=1e-5  # ✅ Increased min_lr to 1e-5
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("resnet50_updated_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,220,419 (100.02 MB)

 Trainable params: 19,579,139 (74.69 MB)

 Non-trainable params: 6,641,280 (25.33 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 246s 7s/step - accuracy: 0.3844 - loss: 21.7175 - val_accuracy: 0.3623 - val_loss: 21.6372 - learning_rate: 1.2000e-04
Epoch 2/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 43s 349ms/step - accuracy: 0.3241 - loss: 21.4622 - val_accuracy: 0.4251 - val_loss: 21.0373 - learning_rate: 1.4400e-04
Epoch 3/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 403ms/step - accuracy: 0.3967 - loss: 20.9314 - val_accuracy: 0.4976 - val_loss: 20.2710 - learning_rate: 1.7280e-04
Epoch 4/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 333ms/step - accuracy: 0.4326 - loss: 20.3179 - val_accuracy: 0.4300 - val_loss: 20.0590 - learning_rate: 2.0736e-04
Epoch 5/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 352ms/step - accuracy: 0.4236 - loss: 19.6135 - val_accuracy: 0.4831 - val_loss: 19.1830 - learning_rate: 2.4883e-04
Epoch 6/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 346ms/step - accuracy: 0.4428 - loss: 18.9438 - val_accuracy: 0.3768 - val_loss: 19.2159 - learning_rate: 2.4883e-04
Epoch 7/25
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 306ms/


Classification Report:
              precision    recall  f1-score   support

          AD       0.81      0.51      0.62        69
        EMCI       0.69      0.58      0.63        69
        LMCI       0.51      0.78      0.62        69

    accuracy                           0.62       207
   macro avg       0.67      0.62      0.62       207
weighted avg       0.67      0.62      0.62       207


Confusion Matrix:
[[35  8 26]
 [ 3 40 26]
 [ 5 10 54]]
Accuracy for AD: 50.72%
Accuracy for EMCI: 57.97%
Accuracy for LMCI: 78.26%


Resnet 50 Batch size 16 and epochs 50

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import mixed_precision

# Enable mixed precision for faster computation
mixed_precision.set_global_policy('mixed_float16')

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 224  # ResNet50 default
IMG_WIDTH = 224
BATCH_SIZE = 16
EPOCHS = 50  # Increased for better feature learning
PATIENCE = 25

# Image Data Generator with proper preprocessing for ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # ADDED
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2  # ADDED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# ✅ Fix: Convert class_weights to dictionary
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# ✅ Fix: Adjust unfreezing (Don't unfreeze too many layers)
for layer in base_model.layers[:-50]:  # Instead of -80
    layer.trainable = False

# Custom Classification Head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Reduce overfitting
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

# ✅ Fix: Remove deprecated LossScaleOptimizer
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr * 1.2  # Gradually increase LR in first 5 epochs
    elif epoch < 10:
        return lr  # Keep stable
    else:
        return lr * 0.95  # Reduce gradually

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=5, factor=0.3, min_lr=1e-5  # ✅ Increased min_lr to 1e-5
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("resnet50_updated_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,220,419 (100.02 MB)

 Trainable params: 19,579,139 (74.69 MB)

 Non-trainable params: 6,641,280 (25.33 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.3471 - loss: 21.9152 - val_accuracy: 0.3913 - val_loss: 21.4202 - learning_rate: 1.2000e-04
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 348ms/step - accuracy: 0.3879 - loss: 21.5219 - val_accuracy: 0.3961 - val_loss: 20.7913 - learning_rate: 1.4400e-04
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 314ms/step - accuracy: 0.3689 - loss: 20.8726 - val_accuracy: 0.4589 - val_loss: 20.4168 - learning_rate: 1.7280e-04
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 335ms/step - accuracy: 0.4634 - loss: 20.2882 - val_accuracy: 0.4541 - val_loss: 19.8026 - learning_rate: 2.0736e-04
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 350ms/step - accuracy: 0.4640 - loss: 19.6174 - val_accuracy: 0.4589 - val_loss: 19.4619 - learning_rate: 2.4883e-04
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 356ms/step - accuracy: 0.4177 - loss: 19.0116 - val_accuracy: 0.4106 - val_loss: 19.6379 - learning_rate: 2.4883e-04
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 328ms/s


Classification Report:
              precision    recall  f1-score   support

          AD       0.80      0.75      0.78        69
        EMCI       0.75      0.77      0.76        69
        LMCI       0.72      0.74      0.73        69

    accuracy                           0.75       207
   macro avg       0.75      0.75      0.75       207
weighted avg       0.75      0.75      0.75       207


Confusion Matrix:
[[52  8  9]
 [ 5 53 11]
 [ 8 10 51]]
Accuracy for AD: 75.36%
Accuracy for EMCI: 76.81%
Accuracy for LMCI: 73.91%


Resnet 50 Batch size 16 and epochs 75

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import mixed_precision

# Enable mixed precision for faster computation
mixed_precision.set_global_policy('mixed_float16')

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 224  # ResNet50 default
IMG_WIDTH = 224
BATCH_SIZE = 16
EPOCHS = 75  # Increased for better feature learning
PATIENCE = 25

# Image Data Generator with proper preprocessing for ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # ADDED
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2  # ADDED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# ✅ Fix: Convert class_weights to dictionary
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# ✅ Fix: Adjust unfreezing (Don't unfreeze too many layers)
for layer in base_model.layers[:-50]:  # Instead of -80
    layer.trainable = False

# Custom Classification Head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Reduce overfitting
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

# ✅ Fix: Remove deprecated LossScaleOptimizer
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr * 1.2  # Gradually increase LR in first 5 epochs
    elif epoch < 10:
        return lr  # Keep stable
    else:
        return lr * 0.95  # Reduce gradually

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=5, factor=0.3, min_lr=1e-5  # ✅ Increased min_lr to 1e-5
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("resnet50_updated_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,220,419 (100.02 MB)

 Trainable params: 19,579,139 (74.69 MB)

 Non-trainable params: 6,641,280 (25.33 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.3508 - loss: 21.8586 - val_accuracy: 0.3575 - val_loss: 21.2844 - learning_rate: 1.2000e-04
Epoch 2/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 313ms/step - accuracy: 0.3496 - loss: 21.3626 - val_accuracy: 0.4541 - val_loss: 20.7168 - learning_rate: 1.4400e-04
Epoch 3/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 379ms/step - accuracy: 0.4578 - loss: 20.8015 - val_accuracy: 0.3816 - val_loss: 20.3035 - learning_rate: 1.7280e-04
Epoch 4/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 355ms/step - accuracy: 0.4358 - loss: 20.1646 - val_accuracy: 0.4106 - val_loss: 19.6027 - learning_rate: 2.0736e-04
Epoch 5/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 357ms/step - accuracy: 0.3916 - loss: 19.5321 - val_accuracy: 0.4976 - val_loss: 18.8402 - learning_rate: 2.4883e-04
Epoch 6/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 12s 383ms/step - accuracy: 0.4504 - loss: 18.7121 - val_accuracy: 0.4734 - val_loss: 17.9933 - learning_rate: 2.4883e-04
Epoch 7/75
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 315ms/s


Classification Report:
              precision    recall  f1-score   support

          AD       0.88      0.75      0.81        69
        EMCI       0.80      0.83      0.81        69
        LMCI       0.74      0.83      0.78        69

    accuracy                           0.80       207
   macro avg       0.81      0.80      0.80       207
weighted avg       0.81      0.80      0.80       207


Confusion Matrix:
[[52  7 10]
 [ 2 57 10]
 [ 5  7 57]]
Accuracy for AD: 75.36%
Accuracy for EMCI: 82.61%
Accuracy for LMCI: 82.61%


Resnet 50 Batch size 16 and epochs 100

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import mixed_precision

# Enable mixed precision for faster computation
mixed_precision.set_global_policy('mixed_float16')

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 224  # ResNet50 default
IMG_WIDTH = 224
BATCH_SIZE = 16
EPOCHS = 100 # Increased for better feature learning
PATIENCE = 25

# Image Data Generator with proper preprocessing for ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # ADDED
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2  # ADDED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# ✅ Fix: Convert class_weights to dictionary
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# ✅ Fix: Adjust unfreezing (Don't unfreeze too many layers)
for layer in base_model.layers[:-50]:  # Instead of -80
    layer.trainable = False

# Custom Classification Head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Reduce overfitting
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

# ✅ Fix: Remove deprecated LossScaleOptimizer
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr * 1.2  # Gradually increase LR in first 5 epochs
    elif epoch < 10:
        return lr  # Keep stable
    else:
        return lr * 0.95  # Reduce gradually

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=5, factor=0.3, min_lr=1e-5  # ✅ Increased min_lr to 1e-5
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("resnet50_updated_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,220,419 (100.02 MB)

 Trainable params: 19,579,139 (74.69 MB)

 Non-trainable params: 6,641,280 (25.33 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.3617 - loss: 21.7789 - val_accuracy: 0.3623 - val_loss: 21.2339 - learning_rate: 1.2000e-04
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 350ms/step - accuracy: 0.3530 - loss: 21.3575 - val_accuracy: 0.4396 - val_loss: 20.7541 - learning_rate: 1.4400e-04
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 336ms/step - accuracy: 0.4216 - loss: 20.9129 - val_accuracy: 0.4251 - val_loss: 20.2114 - learning_rate: 1.7280e-04
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 10s 320ms/step - accuracy: 0.4217 - loss: 20.2365 - val_accuracy: 0.4638 - val_loss: 19.6829 - learning_rate: 2.0736e-04
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 363ms/step - accuracy: 0.4110 - loss: 19.6644 - val_accuracy: 0.4348 - val_loss: 19.3529 - learning_rate: 2.4883e-04
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 359ms/step - accuracy: 0.3685 - loss: 18.9836 - val_accuracy: 0.4155 - val_loss: 18.3318 - learning_rate: 2.4883e-04
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 11s 


Classification Report:
              precision    recall  f1-score   support

          AD       0.85      0.77      0.81        69
        EMCI       0.78      0.72      0.75        69
        LMCI       0.68      0.80      0.73        69

    accuracy                           0.76       207
   macro avg       0.77      0.76      0.76       207
weighted avg       0.77      0.76      0.76       207


Confusion Matrix:
[[53  4 12]
 [ 5 50 14]
 [ 4 10 55]]
Accuracy for AD: 76.81%
Accuracy for EMCI: 72.46%
Accuracy for LMCI: 79.71%


Resnet 50 Batch size 32 and epochs 25

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import mixed_precision

# Enable mixed precision for faster computation
mixed_precision.set_global_policy('mixed_float16')

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 224  # ResNet50 default
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 25  # Increased for better feature learning
PATIENCE = 25

# Image Data Generator with proper preprocessing for ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # ADDED
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2  # ADDED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# ✅ Fix: Convert class_weights to dictionary
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# ✅ Fix: Adjust unfreezing (Don't unfreeze too many layers)
for layer in base_model.layers[:-50]:  # Instead of -80
    layer.trainable = False

# Custom Classification Head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Reduce overfitting
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

# ✅ Fix: Remove deprecated LossScaleOptimizer
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr * 1.2  # Gradually increase LR in first 5 epochs
    elif epoch < 10:
        return lr  # Keep stable
    else:
        return lr * 0.95  # Reduce gradually

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=5, factor=0.3, min_lr=1e-5  # ✅ Increased min_lr to 1e-5
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("resnet50_updated_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_4           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,220,419 (100.02 MB)

 Trainable params: 19,579,139 (74.69 MB)

 Non-trainable params: 6,641,280 (25.33 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.3112 - loss: 21.9627 - val_accuracy: 0.3623 - val_loss: 21.3908 - learning_rate: 1.2000e-04
Epoch 2/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 590ms/step - accuracy: 0.4079 - loss: 21.4718 - val_accuracy: 0.3961 - val_loss: 21.0691 - learning_rate: 1.4400e-04
Epoch 3/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 658ms/step - accuracy: 0.4331 - loss: 21.0535 - val_accuracy: 0.3623 - val_loss: 20.7340 - learning_rate: 1.7280e-04
Epoch 4/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 674ms/step - accuracy: 0.4661 - loss: 20.6983 - val_accuracy: 0.4155 - val_loss: 20.2353 - learning_rate: 2.0736e-04
Epoch 5/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 745ms/step - accuracy: 0.4764 - loss: 20.2030 - val_accuracy: 0.4155 - val_loss: 19.8799 - learning_rate: 2.4883e-04
Epoch 6/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 590ms/step - accuracy: 0.5120 - loss: 19.6359 - val_accuracy: 0.4493 - val_loss: 19.3162 - learning_rate: 2.4883e-04
Epoch 7/25
16/16 ━━━━━━━━━━━━━━━━━━━━ 12s 734ms/st


Classification Report:
              precision    recall  f1-score   support

          AD       0.65      0.70      0.67        69
        EMCI       0.73      0.68      0.71        69
        LMCI       0.65      0.65      0.65        69

    accuracy                           0.68       207
   macro avg       0.68      0.68      0.68       207
weighted avg       0.68      0.68      0.68       207


Confusion Matrix:
[[48  9 12]
 [10 47 12]
 [16  8 45]]
Accuracy for AD: 69.57%
Accuracy for EMCI: 68.12%
Accuracy for LMCI: 65.22%


Resnet 50 Batch size 32 and epochs 50

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import mixed_precision

# Enable mixed precision for faster computation
mixed_precision.set_global_policy('mixed_float16')

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 224  # ResNet50 default
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 50  # Increased for better feature learning
PATIENCE = 25

# Image Data Generator with proper preprocessing for ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # ADDED
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2  # ADDED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# ✅ Fix: Convert class_weights to dictionary
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# ✅ Fix: Adjust unfreezing (Don't unfreeze too many layers)
for layer in base_model.layers[:-50]:  # Instead of -80
    layer.trainable = False

# Custom Classification Head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Reduce overfitting
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

# ✅ Fix: Remove deprecated LossScaleOptimizer
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr * 1.2  # Gradually increase LR in first 5 epochs
    elif epoch < 10:
        return lr  # Keep stable
    else:
        return lr * 0.95  # Reduce gradually

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=5, factor=0.3, min_lr=1e-5  # ✅ Increased min_lr to 1e-5
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("resnet50_updated_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,220,419 (100.02 MB)

 Trainable params: 19,579,139 (74.69 MB)

 Non-trainable params: 6,641,280 (25.33 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.3283 - loss: 21.9606 - val_accuracy: 0.3575 - val_loss: 21.3812 - learning_rate: 1.2000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 595ms/step - accuracy: 0.4078 - loss: 21.5416 - val_accuracy: 0.4058 - val_loss: 21.0182 - learning_rate: 1.4400e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 664ms/step - accuracy: 0.4158 - loss: 21.0670 - val_accuracy: 0.4444 - val_loss: 20.7493 - learning_rate: 1.7280e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 677ms/step - accuracy: 0.4883 - loss: 20.5859 - val_accuracy: 0.4638 - val_loss: 20.1875 - learning_rate: 2.0736e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 675ms/step - accuracy: 0.4233 - loss: 20.2046 - val_accuracy: 0.4444 - val_loss: 19.7314 - learning_rate: 2.4883e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 597ms/step - accuracy: 0.4533 - loss: 19.6703 - val_accuracy: 0.4348 - val_loss: 19.3494 - learning_rate: 2.4883e-04
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 627ms/s

7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 756ms/step



Classification Report:
              precision    recall  f1-score   support

          AD       0.80      0.64      0.71        69
        EMCI       0.67      0.81      0.74        69
        LMCI       0.71      0.71      0.71        69

    accuracy                           0.72       207
   macro avg       0.73      0.72      0.72       207
weighted avg       0.73      0.72      0.72       207


Confusion Matrix:
[[44 13 12]
 [ 5 56  8]
 [ 6 14 49]]
Accuracy for AD: 63.77%
Accuracy for EMCI: 81.16%
Accuracy for LMCI: 71.01%


Resnet 50 Batch size 32 and epochs 75


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import mixed_precision

# Enable mixed precision for faster computation
mixed_precision.set_global_policy('mixed_float16')

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 224  # ResNet50 default
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 75  # Increased for better feature learning
PATIENCE = 25

# Image Data Generator with proper preprocessing for ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # ADDED
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2  # ADDED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# ✅ Fix: Convert class_weights to dictionary
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# ✅ Fix: Adjust unfreezing (Don't unfreeze too many layers)
for layer in base_model.layers[:-50]:  # Instead of -80
    layer.trainable = False

# Custom Classification Head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Reduce overfitting
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

# ✅ Fix: Remove deprecated LossScaleOptimizer
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr * 1.2  # Gradually increase LR in first 5 epochs
    elif epoch < 10:
        return lr  # Keep stable
    else:
        return lr * 0.95  # Reduce gradually

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=5, factor=0.3, min_lr=1e-5  # ✅ Increased min_lr to 1e-5
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("resnet50_updated_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_6           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,220,419 (100.02 MB)

 Trainable params: 19,579,139 (74.69 MB)

 Non-trainable params: 6,641,280 (25.33 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/75
16/16 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.3449 - loss: 21.8615 - val_accuracy: 0.3816 - val_loss: 21.5122 - learning_rate: 1.2000e-04
Epoch 2/75
16/16 ━━━━━━━━━━━━━━━━━━━━ 49s 694ms/step - accuracy: 0.4162 - loss: 21.5252 - val_accuracy: 0.3913 - val_loss: 21.2644 - learning_rate: 1.4400e-04
Epoch 3/75
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 586ms/step - accuracy: 0.4630 - loss: 21.0828 - val_accuracy: 0.4203 - val_loss: 20.6806 - learning_rate: 1.7280e-04
Epoch 4/75
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 693ms/step - accuracy: 0.4788 - loss: 20.5941 - val_accuracy: 0.4396 - val_loss: 20.4948 - learning_rate: 2.0736e-04
Epoch 5/75
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 674ms/step - accuracy: 0.4968 - loss: 20.0986 - val_accuracy: 0.4734 - val_loss: 19.8215 - learning_rate: 2.4883e-04
Epoch 6/75
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 675ms/step - accuracy: 0.5294 - loss: 19.5392 - val_accuracy: 0.4734 - val_loss: 19.1616 - learning_rate: 2.4883e-04
Epoch 7/75
16/16 ━━━━━━━━━━━━━━━━━━━━ 9s 590ms/st

7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 977ms/step



Classification Report:
              precision    recall  f1-score   support

          AD       0.89      0.72      0.80        69
        EMCI       0.82      0.80      0.81        69
        LMCI       0.73      0.88      0.80        69

    accuracy                           0.80       207
   macro avg       0.81      0.80      0.80       207
weighted avg       0.81      0.80      0.80       207


Confusion Matrix:
[[50  6 13]
 [ 4 55 10]
 [ 2  6 61]]
Accuracy for AD: 72.46%
Accuracy for EMCI: 79.71%
Accuracy for LMCI: 88.41%


Resnet 50 Batch size 32 and epochs 100

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import mixed_precision

# Enable mixed precision for faster computation
mixed_precision.set_global_policy('mixed_float16')

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# Image dimensions and batch size
IMG_HEIGHT = 224  # ResNet50 default
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 100  # Increased for better feature learning
PATIENCE = 25

# Image Data Generator with proper preprocessing for ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # ADDED
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.6, 1.4],
    channel_shift_range=0.2  # ADDED
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Compute class weights to handle class imbalance
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# ✅ Fix: Convert class_weights to dictionary
class_weights = {i: weight for i, weight in enumerate(class_weights)}

# Load ResNet50 with pre-trained weights, excluding top layers
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)

# ✅ Fix: Adjust unfreezing (Don't unfreeze too many layers)
for layer in base_model.layers[:-50]:  # Instead of -80
    layer.trainable = False

# Custom Classification Head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Reduce overfitting
    layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')
])

# ✅ Fix: Remove deprecated LossScaleOptimizer
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Learning rate scheduler with warm-up
def scheduler(epoch, lr):
    if epoch < 5:
        return lr * 1.2  # Gradually increase LR in first 5 epochs
    elif epoch < 10:
        return lr  # Keep stable
    else:
        return lr * 0.95  # Reduce gradually

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Callbacks
lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=5, factor=0.3, min_lr=1e-5  # ✅ Increased min_lr to 1e-5
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE, restore_best_weights=True
)

# Model summary
model.summary()

# Train the model
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=[lr_reduction, early_stopping, lr_scheduler],
    class_weight=class_weights,
    verbose=1
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Predictions and class-wise metrics
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=-1)
y_true = test_generator.classes

# Classification Report
class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion Matrix
print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

# Class-wise Accuracy
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for label, acc in zip(class_labels, class_accuracies):
    print(f"Accuracy for {label}: {acc * 100:.2f}%")

# Save the model
model.save("resnet50_updated_model.h5")


Found 483 images belonging to 3 classes.
Found 207 images belonging to 3 classes.


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_7           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 2048)                │           8,192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,220,419 (100.02 MB)

 Trainable params: 19,579,139 (74.69 MB)

 Non-trainable params: 6,641,280 (25.33 MB)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.2996 - loss: 21.9221 - val_accuracy: 0.3382 - val_loss: 21.7051 - learning_rate: 1.2000e-04
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 728ms/step - accuracy: 0.3387 - loss: 21.6013 - val_accuracy: 0.3671 - val_loss: 21.1417 - learning_rate: 1.4400e-04
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 603ms/step - accuracy: 0.4238 - loss: 21.1999 - val_accuracy: 0.4010 - val_loss: 20.7904 - learning_rate: 1.7280e-04
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 636ms/step - accuracy: 0.4813 - loss: 20.7112 - val_accuracy: 0.4106 - val_loss: 20.3909 - learning_rate: 2.0736e-04
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 675ms/step - accuracy: 0.4463 - loss: 20.3208 - val_accuracy: 0.4783 - val_loss: 19.9777 - learning_rate: 2.4883e-04
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 698ms/step - accuracy: 0.5120 - loss: 19.6963 - val_accuracy: 0.4589 - val_loss: 19.5095 - learning_rate: 2.4883e-04
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 


Classification Report:
              precision    recall  f1-score   support

          AD       0.86      0.72      0.79        69
        EMCI       0.76      0.86      0.80        69
        LMCI       0.77      0.80      0.79        69

    accuracy                           0.79       207
   macro avg       0.80      0.79      0.79       207
weighted avg       0.80      0.79      0.79       207


Confusion Matrix:
[[50  8 11]
 [ 5 59  5]
 [ 3 11 55]]
Accuracy for AD: 72.46%
Accuracy for EMCI: 85.51%
Accuracy for LMCI: 79.71%
